# Importing Libraries

In [1]:
# For working with images and displaying them
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random

# For File Handling
import os
import shutil as sh
from shutil import copyfile

# For Splitting dataset
from sklearn.model_selection import train_test_split

# For creating a progress bar
from tqdm.auto import tqdm


# For integrating YOLO with WandB.ai
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient() 
import gc

# For YOLO model saving
import yaml

## Installing Yolov5

In [2]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12390, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 12390 (delta 1), reused 7 (delta 1), pack-reused 12383
Receiving objects: 100% (12390/12390), 11.56 MiB | 30.44 MiB/s, done.
Resolving deltas: 100% (8620/8620), done.
/kaggle/working/yolov5
Note: you may need to restart the kernel to use updated packages.
/kaggle/working
Setup complete. Using torch 1.7.1+cu110 (Tesla P100-PCIE-16GB)


## Setting Up WandB

In [3]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 

personal_key_for_api = user_secrets.get_secret("wandb")

! wandb login $personal_key_for_api

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Combine category folders

In [4]:
combined_path = "combi"
train_label_path = combined_path + "/labels/train/"
train_image_path = combined_path + "/images/train/"
val_label_path = combined_path + "/labels/val/"
val_image_path = combined_path + "/images/val/"
os.mkdir(combined_path)
os.mkdir(combined_path+"/labels")
os.mkdir(combined_path+"/images")
os.mkdir(train_image_path)
os.mkdir(train_label_path)
os.mkdir(val_label_path)
os.mkdir(val_image_path)

input_images_path = "../input/african-wildlife/"
dirnames = next(os.walk(input_images_path), (None, None, []))[1]

for dir in dirnames:
    print(dir)
    filenames = next(os.walk(input_images_path + dir), (None, None, []))[2]  # [] if no file
    for file in filenames:
        if ".txt" in file:
            if(random.uniform(0, 1) > .33):
                try:
                    sh.copy(input_images_path + dir + "/" + file, train_label_path + file)
                    sh.copy(input_images_path + dir + "/" + file.replace('.txt', '.jpg'), train_image_path + file.replace('.txt', '.jpg'))
                except:
                    if os.path.exists(train_label_path + file):
                        os.remove(train_label_path + file)
                    if os.path.exists(train_image_path + file.replace('.txt', '.jpg')):
                        os.remove(train_image_path + file.replace('.txt', '.jpg'))
            else:
                try:
                    sh.copy(input_images_path + dir + "/" + file, val_label_path + file)
                    sh.copy(input_images_path + dir + "/" + file.replace('.txt', '.jpg'), val_image_path + file.replace('.txt', '.jpg'))
                except:
                    if os.path.exists(val_label_path + file):
                        os.remove(val_label_path + file)
                    if os.path.exists(val_image_path + file.replace('.txt', '.jpg')):
                        os.remove(val_image_path + file.replace('.txt', '.jpg'))
            
                

buffalo
elephant
zebra
rhino


## YAML File

In [5]:
# Create .yaml file 

data_yaml = dict(
    train = '../combi/images/train',
    val = '../combi/images/val',
    nc = 4,
    names = ['buffalo', 'elephant', 'rhino', 'zebra']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
    
%cat yolov5/data/data.yaml

{names: [buffalo, elephant, rhino, zebra], nc: 4, train: ../combi/images/train, val: ../combi/images/val}


## Checking the number of Workers

In [6]:
import multiprocessing
multiprocessing.cpu_count()

2

# Training YOLOv5x6 Model

In [7]:
%cd yolov5/
!python train.py --img 256 --batch 2 --epochs 50 --data data.yaml --weights yolov5x6.pt --workers 2 --cache --project African_Wildlife

/kaggle/working/yolov5
wandb: Currently logged in as: vermaayush680 (use `wandb login --relogin` to force relogin)
2022-04-01 15:22:21.938636: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-01 15:22:27.036295: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.12.11
wandb: Run data is saved locally in /kaggle/working/yolov5/wandb/run-20220401_152225-bcybknf5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lilac-disco-5
wandb: ⭐️ View project at https://wandb.ai/vermaayush680/African_Wildlife
wandb: 🚀 View run at https://wandb.ai/vermaayush680/African_Wildlife/runs/bcybknf5
100%|████████████████████████████████████████| 270M/270M [00:03<00:00, 72.4MB/s]

train: Scanning '/kaggle/working/yolov5/../combi/labels/train' images and labels
train: Caching images (0.0GB ram): 100%|██████████|